In [48]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier, cv, plot_importance
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.metrics import classification_report

In [49]:
train = pd.read_csv('/Users/wanranli/Downloads/!Capstone Project 2/datasets/train_ML.csv')
train.head()

,Unnamed: 0,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,0,2987000,0,86400,68.5,4,13926,-999.0,150.0,1,...,136,-999.0,461,4,2,2,2,2,2,2740
1,1,2987001,0,86401,29.0,4,2755,404.0,150.0,2,...,136,-999.0,461,4,2,2,2,2,2,2740
2,2,2987002,0,86469,59.0,4,4663,490.0,150.0,4,...,136,-999.0,461,4,2,2,2,2,2,2740
3,3,2987003,0,86499,50.0,4,18132,567.0,150.0,2,...,136,-999.0,461,4,2,2,2,2,2,2740
4,4,2987004,0,86506,50.0,1,4497,514.0,150.0,2,...,162,32.0,268,3,1,0,1,1,1,1565


In [50]:
train.drop('Unnamed: 0',axis=1,inplace=True)
train.head(3)

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987000,0,86400,68.5,4,13926,-999.0,150.0,1,142.0,...,136,-999.0,461,4,2,2,2,2,2,2740
1,2987001,0,86401,29.0,4,2755,404.0,150.0,2,102.0,...,136,-999.0,461,4,2,2,2,2,2,2740
2,2987002,0,86469,59.0,4,4663,490.0,150.0,4,166.0,...,136,-999.0,461,4,2,2,2,2,2,2740


In [244]:
train.shape

(590540, 352)

In [245]:
df = train.head(100000)

In [246]:
df.isFraud.value_counts()

0    97439
1     2561
Name: isFraud, dtype: int64

In [247]:
df.isFraud.value_counts(normalize=True)

0    0.97439
1    0.02561
Name: isFraud, dtype: float64

In [248]:
features = ['TransactionDT', 'TransactionAmt', 'ProductCD', 'card1', 'card2', 'card3', 'card4', 'card5', 'card6',
       'addr1', 'addr2', 'dist1', 'P_emaildomain', 'R_emaildomain', 'C1',
       'C2', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12',
       'C13', 'C14', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D8', 'D9',
       'D10', 'D11', 'D12', 'D13', 'D14', 'D15', 'M1', 'M2', 'M3', 'M4',
       'M5', 'M6', 'M7', 'M8', 'M9', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6',
       'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V15', 'V16', 'V17',
       'V18', 'V19', 'V20', 'V21', 'V22', 'V29', 'V30', 'V31', 'V32',
       'V33', 'V34', 'V35', 'V36', 'V37', 'V38', 'V39', 'V40', 'V41',
       'V42', 'V43', 'V44', 'V45', 'V46', 'V47', 'V48', 'V49', 'V50',
       'V51', 'V52', 'V53', 'V54', 'V56', 'V57', 'V58', 'V59', 'V60',
       'V61', 'V62', 'V63', 'V64', 'V69', 'V70', 'V71', 'V72', 'V73',
       'V74', 'V75', 'V76', 'V78', 'V79', 'V80', 'V81', 'V82', 'V83',
       'V84', 'V85', 'V87', 'V90', 'V91', 'V92', 'V93', 'V94', 'V95',
       'V96', 'V97', 'V99', 'V100', 'V126', 'V127', 'V128', 'V130',
       'V131', 'V138', 'V139', 'V140', 'V141', 'V142', 'V143', 'V144',
       'V145', 'V146', 'V147', 'V148', 'V149', 'V150', 'V151', 'V152',
       'V153', 'V154', 'V155', 'V156', 'V157', 'V158', 'V159', 'V160',
       'V161', 'V162', 'V163', 'V164', 'V165', 'V166', 'V167', 'V168',
       'V169', 'V170', 'V171', 'V172', 'V173', 'V174', 'V175', 'V176',
       'V177', 'V178', 'V179', 'V180', 'V181', 'V182', 'V183', 'V184',
       'V185', 'V186', 'V187', 'V188', 'V189', 'V190', 'V191', 'V192',
       'V193', 'V194', 'V195', 'V196', 'V197', 'V198', 'V199', 'V200',
       'V201', 'V202', 'V203', 'V204', 'V205', 'V206', 'V207', 'V208',
       'V209', 'V210', 'V211', 'V212', 'V213', 'V214', 'V215', 'V216',
       'V217', 'V218', 'V219', 'V220', 'V221', 'V222', 'V223', 'V224',
       'V225', 'V226', 'V227', 'V228', 'V229', 'V230', 'V231', 'V232',
       'V233', 'V234', 'V235', 'V236', 'V237', 'V238', 'V239', 'V240',
       'V241', 'V242', 'V243', 'V244', 'V245', 'V246', 'V247', 'V248',
       'V249', 'V250', 'V251', 'V252', 'V253', 'V254', 'V255', 'V256',
       'V257', 'V258', 'V259', 'V260', 'V261', 'V262', 'V263', 'V264',
       'V265', 'V266', 'V267', 'V268', 'V269', 'V270', 'V271', 'V272',
       'V273', 'V274', 'V275', 'V276', 'V277', 'V278', 'V279', 'V280',
       'V282', 'V283', 'V285', 'V287', 'V288', 'V289', 'V291', 'V292',
       'V294', 'V302', 'V303', 'V304', 'V306', 'V307', 'V308', 'V310',
       'V312', 'V313', 'V314', 'V315', 'V317', 'V322', 'V323', 'V324',
       'V325', 'V326', 'V327', 'V328', 'V329', 'V330', 'V331', 'V332',
       'V333', 'V334', 'V335', 'V336', 'V337', 'V338', 'V339', 'id_01',
       'id_02', 'id_03', 'id_04', 'id_05', 'id_06', 'id_09', 'id_10',
       'id_11', 'id_12', 'id_13', 'id_14', 'id_15', 'id_16', 'id_17',
       'id_19', 'id_20', 'id_28', 'id_29', 'id_30', 'id_31', 'id_32',
       'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38', 'DeviceType',
       'DeviceInfo']

In [249]:
X = df[features]
y = df.isFraud

# split training and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [250]:
parameters = {'min_child_weight': [0, 1], 'scale_pos_weight':[0.95, 0.9]}

In [251]:
# instantiate a classifier
xg_clf = XGBClassifier(objective = 'binary:logistic', random_state = 42, silent = 1, n_estimators=10, max_depth= 8)

In [255]:
# RandomSearch
grid_search = GridSearchCV(param_grid= parameters, estimator = xg_clf, cv=3, scoring='accuracy')
grid_search.fit(X_train,y_train)

GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1, gamma=0,
                                     learning_rate=0.1, max_delta_step=0,
                                     max_depth=8, min_child_weight=1,
                                     missing=None, n_estimators=10, n_jobs=1,
                                     nthread=None, objective='binary:logistic',
                                     random_state=42, reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, seed=None, silent=1,
                                     subsample=1, verbosity=1),
             iid='warn', n_jobs=None,
             param_grid={'min_child_weight': [0, 1],
                         'scale_pos_weight': [0.95, 0.9]},
             pre_dispatch='2*n_jobs', refit=True, return_t

In [256]:
# Print best parameters and results
print(grid_search.best_params_)
print(grid_search.best_score_)

{'min_child_weight': 0, 'scale_pos_weight': 0.9}
0.9808


In [257]:
# optimized parameters1
optimized_parameters = {'objective':'binary:logistic',
                        'min_child_weight': 1,
                        'scale_pos_weight': 0.9,
                        'n_estimators': 500,
                        'num_leaves': 20,
                        'min_child_samples': 79,
                        'max_depth': 13,
                        "subsample_freq": 3,
                        "subsample": 0.9,
                        "bagging_seed": 11,
                        'learning_rate':0.03     
                       }

In [258]:
# instantiate a classifier
xg_clf_opt = XGBClassifier(**optimized_parameters)
xg_clf_opt.fit(X_train, y_train)

XGBClassifier(bagging_seed=11, base_score=0.5, booster='gbtree',
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              gamma=0, learning_rate=0.03, max_delta_step=0, max_depth=13,
              min_child_samples=79, min_child_weight=1, missing=None,
              n_estimators=500, n_jobs=1, nthread=None, num_leaves=20,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=0.9, seed=None, silent=None,
              subsample=0.9, subsample_freq=3, verbosity=1)

In [259]:
# predict the labels
y_opt_pred = xg_clf_opt.predict(X_train)

# score model
score = accuracy_score(y_train, y_opt_pred)
print("The accuracy of the optimized classifier is {}".format(round(score, 5)))

The accuracy of the optimized classifier is 0.99889


In [260]:
# use classification_report to evaluate the performance
report = classification_report(y_train, y_opt_pred, output_dict=True)
report

{'0': {'precision': 0.9988570926194558,
  'recall': 1.0,
  'f1-score': 0.9994282195636875,
  'support': 68169},
 '1': {'precision': 1.0,
  'recall': 0.957400327689787,
  'f1-score': 0.9782366071428571,
  'support': 1831},
 'accuracy': 0.9988857142857143,
 'macro avg': {'precision': 0.9994285463097279,
  'recall': 0.9787001638448936,
  'f1-score': 0.9888324133532723,
  'support': 70000},
 'weighted avg': {'precision': 0.9988869878110811,
  'recall': 0.9988857142857143,
  'f1-score': 0.9988739075302225,
  'support': 70000}}

In [261]:
df_report = pd.DataFrame(report).transpose()
df_report

,precision,recall,f1-score,support
0,0.998857,1.000000,0.999428,68169.000000
1,1.000000,0.957400,0.978237,1831.000000
accuracy,0.998886,0.998886,0.998886,0.998886
macro avg,0.999429,0.978700,0.988832,70000.000000
weighted avg,0.998887,0.998886,0.998874,70000.000000


In [262]:
# predict the labels of test data by using the optimized classifier
y_opt_test = xg_clf_opt.predict(X_test)

# score model
test_score = accuracy_score(y_test, y_opt_test)
print('The accuracy of the optimized classifier for test data is.{}'.format(round(test_score, 5)))

The accuracy of the optimized classifier for test data is.0.98833


In [263]:
# use classification_report to evaluate the performance
report_test = classification_report(y_test, y_opt_test, output_dict=True)
report_test

{'0': {'precision': 0.9884805080737789,
  'recall': 0.9996925179364538,
  'f1-score': 0.994054898763419,
  'support': 29270},
 '1': {'precision': 0.9773869346733668,
  'recall': 0.5328767123287671,
  'f1-score': 0.6897163120567377,
  'support': 730},
 'accuracy': 0.9883333333333333,
 'macro avg': {'precision': 0.9829337213735728,
  'recall': 0.7662846151326104,
  'f1-score': 0.8418856054100783,
  'support': 30000},
 'weighted avg': {'precision': 0.9882105644543688,
  'recall': 0.9883333333333333,
  'f1-score': 0.9866493264868897,
  'support': 30000}}

In [264]:
df_report_test = pd.DataFrame(report_test).transpose()
df_report_test

,precision,recall,f1-score,support
0,0.988481,0.999693,0.994055,29270.000000
1,0.977387,0.532877,0.689716,730.000000
accuracy,0.988333,0.988333,0.988333,0.988333
macro avg,0.982934,0.766285,0.841886,30000.000000
weighted avg,0.988211,0.988333,0.986649,30000.000000
